In [1]:
import random

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder


In [2]:
columns = [
    'Transaction_Unique_Identifier',  # *
    'Price',
    'Date_of_Transfer',
    'Post_Code',  # *
    'Property_Type',
    'Old_New',
    'Duration',
    'PAON',  # *
    'SAON',  # *
    'Street',  # *
    'Locality',  # *
    'Town_City',
    'District',
    'County',
    'PPDCategory_Type',
    'Record_Status'
]
original_df = pd.read_csv('data/pp-2021.csv', names=columns)
original_df['Date_of_Transfer'] = pd.to_datetime(original_df['Date_of_Transfer'])

In [3]:
print(original_df.shape)
original_df.head()

(554279, 16)


,Transaction_Unique_Identifier,Price,Date_of_Transfer,Post_Code,Property_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPDCategory_Type,Record_Status
0,{C8A3A576-4E89-0425-E053-6C04A8C0947B},140000,2021-03-18,BB1 8BT,T,N,F,26,NaN,PARK AVENUE,NaN,BLACKBURN,BLACKBURN WITH DARWEN,BLACKBURN WITH DARWEN,A,A
1,{C8A3A576-4E8A-0425-E053-6C04A8C0947B},80500,2021-06-18,FY7 7ND,T,N,F,3,NaN,NORTHFLEET AVENUE,NaN,FLEETWOOD,WYRE,LANCASHIRE,B,A
2,{C8A3A576-4E8B-0425-E053-6C04A8C0947B},220000,2021-06-18,PR26 7AW,D,N,F,91,NaN,FERNLEIGH,NaN,LEYLAND,SOUTH RIBBLE,LANCASHIRE,A,A
3,{C8A3A576-4E8C-0425-E053-6C04A8C0947B},166000,2021-07-14,PR4 3UN,S,N,F,18,NaN,PREESE GARDENS,ELSWICK,PRESTON,FYLDE,LANCASHIRE,A,A
4,{C8A3A576-4E8E-0425-E053-6C04A8C0947B},75000,2021-04-26,BB10 4LF,S,N,F,88,NaN,WYCOLLER AVENUE,NaN,BURNLEY,BURNLEY,LANCASHIRE,A,A


In [4]:
(original_df.isna().sum() / original_df.shape[0]) * 100

Transaction_Unique_Identifier     0.000000
Price                             0.000000
Date_of_Transfer                  0.000000
Post_Code                         0.318071
Property_Type                     0.000000
Old_New                           0.000000
Duration                          0.000000
PAON                              0.000000
SAON                             89.949466
Street                            2.046262
Locality                         61.628169
Town_City                         0.000000
District                          0.000000
County                            0.000000
PPDCategory_Type                  0.000000
Record_Status                     0.000000
dtype: float64

In [5]:
dropped_columns = [
    'Transaction_Unique_Identifier',
    'SAON',
    'PAON',
    'Street',
    'Locality',
    'Date_of_Transfer',
    'Record_Status',
]

base_df = original_df.drop(dropped_columns, axis=1)
base_df

,Price,Post_Code,Property_Type,Old_New,Duration,Town_City,District,County,PPDCategory_Type
0,140000,BB1 8BT,T,N,F,BLACKBURN,BLACKBURN WITH DARWEN,BLACKBURN WITH DARWEN,A
1,80500,FY7 7ND,T,N,F,FLEETWOOD,WYRE,LANCASHIRE,B
2,220000,PR26 7AW,D,N,F,LEYLAND,SOUTH RIBBLE,LANCASHIRE,A
3,166000,PR4 3UN,S,N,F,PRESTON,FYLDE,LANCASHIRE,A
4,75000,BB10 4LF,S,N,F,BURNLEY,BURNLEY,LANCASHIRE,A
...,...,...,...,...,...,...,...,...,...
554274,299995,CF64 5WE,D,Y,F,PENARTH,THE VALE OF GLAMORGAN,THE VALE OF GLAMORGAN,A
554275,250000,LL17 0PY,D,N,F,ST ASAPH,DENBIGHSHIRE,DENBIGHSHIRE,A
554276,278995,NP12 2QU,D,Y,F,BLACKWOOD,CAERPHILLY,CAERPHILLY,A
554277,310000,CF64 5WD,D,Y,F,PENARTH,THE VALE OF GLAMORGAN,THE VALE OF GLAMORGAN,A


## Select Resolution



In [6]:
geo_dropped_columns = [
    'Post_Code',
    'Town_City',
    'District',
    'County',
]

geo_res_df = base_df.drop(geo_dropped_columns, axis=1)
geo_res_df

,Price,Property_Type,Old_New,Duration,PPDCategory_Type
0,140000,T,N,F,A
1,80500,T,N,F,B
2,220000,D,N,F,A
3,166000,S,N,F,A
4,75000,S,N,F,A
...,...,...,...,...,...
554274,299995,D,Y,F,A
554275,250000,D,N,F,A
554276,278995,D,Y,F,A
554277,310000,D,Y,F,A


## One Hot Encoding

In [11]:
encoded_columns = [
    'Property_Type',
    'Old_New',
    'Duration',
    'PPDCategory_Type',
]

enc_df = geo_res_df.copy()
for enc_col in encoded_columns:
    enc_df = pd.concat([
            enc_df, 
            pd.get_dummies(
                enc_df[enc_col], prefix=enc_col,
                drop_first=True, sparse=False
            )
        ],
        axis=1
    )
    enc_df = enc_df.drop(enc_col, axis=1)
enc_df

,Price,Property_Type_F,Property_Type_O,Property_Type_S,Property_Type_T,Old_New_Y,Duration_L,PPDCategory_Type_B
0,140000,0,0,0,1,0,0,0
1,80500,0,0,0,1,0,0,1
2,220000,0,0,0,0,0,0,0
3,166000,0,0,1,0,0,0,0
4,75000,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...
554274,299995,0,0,0,0,1,0,0
554275,250000,0,0,0,0,0,0,0
554276,278995,0,0,0,0,1,0,0
554277,310000,0,0,0,0,1,0,0


In [14]:
enc_df.to_csv('data/ppd2.csv', index=False)